## 06 군집화 실습 - 고객 세그먼테이션
### 고객 세그먼테이션의 정의와 기법

- 가장 기본적인 고객 분석 요소인 RFM 기법 활용
> - RECENCY (R) : 가장 최근 상품 구입일에서 오늘까지의 기간
> - FREQUENCY (F) : 상품 구매 횟수
> - MONETARY VALUE (M) : 총 구매 금액

### 데이터 세트 로딩과 데이터 클렌징

- 활용 데이터 세트 : http://archive.ics.uci.edu/ml/datasets/online+retail

In [2]:
import pandas as pd
import datetime
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

retail_df = pd.read_excel(io='./data/Online Retail.xlsx')
retail_df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom


**칼럼 설명**
- InovoiceNo : 주문번호. 'C'로 시작하는 것은 취소 주문
- StockCode : 제품 코드(Item Code)
- Description : 제품 설명
- Quantity : 주문 건수
- InvoiceDate : 주문 일자
- UnitPrice : 제품 단가
- CustomerID : 고객 번호
- Country : 국가명(주문 고객의 국적)

In [3]:
# 데이터 세트의 전체 건수, 칼럼 타입, Null 개수 확인
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


해석
> 전체 데이터는 541,909개. CustomerID의 Null값이 많음. 고객 세그먼테이션을 수행하므로 ID가 없는 데이터는 불필요하기에 삭제
> 
> 오류 데이터 삭제 : Quantity 또는 UnitPrice가 0보다 작은 경우. (Quantity가 0보다 작은 경우는 오류 데이터라기보다는 반환을 뜻하는 값이며, InvoiceNo의 앞자리가 'C'로 시작한다. 하지만 분석의 효율을 위해 삭제한다.)

In [5]:
retail_df = retail_df[retail_df['Quantity']>0]
retail_df = retail_df[retail_df['UnitPrice']>0]
retail_df = retail_df[retail_df['CustomerID'].notnull()]
print(retail_df.shape)
retail_df.isnull().sum()

(397884, 8)


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [7]:
# Country 칼럼 확인
retail_df['Country'].value_counts()

United Kingdom          354321
Germany                   9040
France                    8341
EIRE                      7236
Spain                     2484
Netherlands               2359
Belgium                   2031
Switzerland               1841
Portugal                  1462
Australia                 1182
Norway                    1071
Italy                      758
Channel Islands            748
Finland                    685
Cyprus                     614
Sweden                     451
Austria                    398
Denmark                    380
Poland                     330
Japan                      321
Israel                     248
Unspecified                244
Singapore                  222
Iceland                    182
USA                        179
Canada                     151
Greece                     145
Malta                      112
United Arab Emirates        68
European Community          60
RSA                         57
Lebanon                     45
Lithuani

영국이 대부분이므로, 영국에 한정해 분석하기로 한다.

In [8]:
retail_df = retail_df[retail_df['Country']=='United Kingdom']
print(retail_df.shape)

(354321, 8)


### RFM 기반 데이터 가공
- 'UnitPrice'와 'Quantity'를 곱해서 주문 금액 데이터 만들기
- CustomerID를 float형에서 int형으로 변경 (식별성을 높이기 위해)

In [9]:
retail_df['sale_amount'] = retail_df['Quantity']*retail_df['UnitPrice']
retail_df['CustomerID'] = retail_df['CustomerID'].astype(int)

해당 데이터세트는 주문횟수와 주문금액이 압도적으로 특정 고객에게 많은 특성을 가진다. (개인 고객의 주문과 소매점 주문이 포함돼 있기 때문)
- Top 5 주문 건수와 주문 금액을 가진 고객 데이터를 추출해보기

In [11]:
print(retail_df['CustomerID'].value_counts().head(5))
print(retail_df.groupby('CustomerID')['sale_amount'].sum().sort_values(ascending=False)[:5])

17841    7847
14096    5111
12748    4595
14606    2700
15311    2379
Name: CustomerID, dtype: int64
CustomerID
18102    259657.30
17450    194550.79
16446    168472.50
17511     91062.38
16029     81024.84
Name: sale_amount, dtype: float64
